In [48]:
from config import *
#from alpaca_trade_api import StreamConn
import websocket, json, asyncio, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller,kpss
from statsmodels.tools.eval_measures import mse
import warnings
import tensorflow as tf 
import statsmodels.api as sm
from pulp import LpVariable,LpProblem,LpMinimize,value
warnings.filterwarnings("ignore")

<h1> The Strategy: </h1> 
<p> The strategy chooses a high volume mean reverting stock(using ADF and KPSS) from the previous day's quote data and trains an NN to determine the mean price change within the next time step, if the delta change is above threshold, buy/sell accordingly. Trading occurs during specific windows: 9am to 11:00 pm and  1pm to 3pm</p>

<h4> Correction: Trade SPY by default</h4>

<ol> 
    <strong> Logic </strong> 
    <li>loop through high volume stocks list: determine stationarity via ADF and KPSS. Then determine backtested accuracy of NN forecasting mean price change within forecast window (on validation set)   </li>
    <li> during trading hours: monitor and forecast price change with every new incoming quote, if abs(mean price change) > 0.2 ticks, buy/sell with take profit accordingly </li> 
    <li> close any open positions at end of trading session </li> 
    
</ol>

<ul> <strong>  NN Architecture: </strong> 
    <li> hyperparameters:- forecast window: k,lag window L, neurons, LSTM/FC layers (previous n days data (n = 2)  </li> 
    <li> X :-  (m x 3) features: 1. rolling_sum(oi/bid_ask_spread) over lag window, 2. rolling_sum(oir/bid_ask_spread), 3. mpb/bid_ask_spread </li> 
    <li> Y :- (1 x m) forecast price over forecast window </li> 
</ul> 


reference: https://pdfs.semanticscholar.org/cf7e/9d9960be215cbc5a0b9f39d4c50879568be3.pdf

<h2> Formulas: </h2>


Order Imbalance as: 

<img src="images/voi_imbalance.png" width="500"/>

Measure Volume Order imbalance ratio as: 

<img src="images/oir.png" width="200"/>

Develop Linear Model from previous Day's prices with: 
<img src="images/linear_model.png" width="500"/>

mean price basis 
<img src="images/mpb.png" width="500"/>
<img src="images/mpb_explanation.png" width="500"/>

Average Trade Price: 
<img src="images/tp.png" width="500"/>

<p> <strong> Important note: volume is cumulative volume </strong> </p>

default values for linear model: k=5



In [ ]:


def on_open(ws):
    print("stream connection opened")
    auth_data = {
        "action": "auth",
        "params": api_key_id
    }

    ws.send(json.dumps(auth_data))

    channel_data = {
        "action": "subscribe",
        "params": 'Q.SPY'
    }
    ws.send(json.dumps(channel_data))
    
def on_message(ws,data):
    data = json.loads(data)
    for quote in data:
        for key,value in quote.items():
            if key == 't': value = time.strftime('%Y-%m-%d %H:%M:%S.%Z', time.localtime(value/1000))
            print('{}: {} '.format(key,value))
            
def on_close(ws):
    print('closed streaming_connection')

socket = "wss://alpaca.socket.polygon.io/stocks"

ws = websocket.WebSocketApp(socket, on_open=on_open, on_message=on_message, on_close=on_close)
ws.run_forever()

In [ ]:

''' sample quote '''
ev: Q #quote 
sym: SPY #ticker 
c: 1 # idk
bx: 12 # bid exchange 
ax: 11  #ask exchange 
bp: 301.52 #bid price
ap: 301.54 # ask price 
bs: 10 #bid size 
as: 4  #ask size 
t: 2020-06-11 15:01:55 # time to the nearest second 
z: 1 # tape ID 



In [17]:
class network:
    def __init__(self,layers = [3,1]):
        self.dimensions = layers
        self.model = tf.keras.Sequential()
        for layer in layers[:-1]: 
            self.model.add(tf.keras.layers.Dense(layer))
        self.model.add(tf.keras.layers.Dense(layers[-1]))
        self.loss_fn = tf.keras.losses.MeanSquaredError()
        self.model.compile(optimizer = 'adam',
              loss=self.loss_fn,
             metrics = ['accuracy'])
        
    def train_model(self,X_train,Y_train,num_epochs):
        self.model.fit(X_train,Y_train,epochs = num_epochs)
        
    def evaluate_model(self,X_test,Y_test):
        return self.model.evaluate(X_test,Y_test,verbose =2) # returns test_loss,test_acc

    
def backtest_daterange(end_date,lookback):
    end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d')
    return [(end_date - datetime.timedelta(days=i)).strftime('%Y-%m-%d') for i in reversed(range(lookback)) if (end_date - datetime.timedelta(days=i)).isoweekday() in range(1,6)] 

def turnover_and_mtp_data(asset,date):
    data =  api.polygon.historic_trades_v2(asset, date).df.drop(
    ['participant_timestamp',
    'trf_timestamp',
    'sequence_number',
    'id',
    'exchange',
    'conditions',
    'tape'],
    axis=1).rename(columns = {'size': 'vol'})
    turnover = pd.DataFrame()
    turnover['volume'] = data.vol.resample('1S').sum()
    turnover['price'] = data.price.resample('1S').mean()
    turnover['turnover'] = turnover['price'] * turnover['volume']
    turnover['mtp'] = (turnover['turnover'] - turnover['turnover'].shift(1))/(turnover['volume'] - turnover['volume'].shift(1))
    turnover.loc[turnover['volume'] == turnover['volume'].shift(1),'mtp'] = turnover.loc[turnover['volume'] == turnover['volume'].shift(1),'mtp'].shift(1)
    return turnover


def get_historic_quotes(asset,date):
    df = api.polygon.historic_quotes_v2(asset,date).df.sort_index().drop(
        ['trf_timestamp',
        'sequence_number',
        'conditions',
        'indicators',
        'bid_exchange',
        'ask_exchange',
        'tape',
        'participant_timestamp'], 
        axis = 1)
    #sample_df = df.resample('1min').mean()
    #df.index = df.index.strftime('%m/%d/%Y %H:%M:%S')
    #sample_df.index = sample_df.index.strftime('%m/%d/%Y %H:%M:%S')
    #print(df)
    #print(sample_df)
    resampled = pd.DataFrame()
    resampled['bid_price'] = df.bid_price.resample('1S').mean()
    resampled['bid_size'] = df.bid_size.resample('1S').sum()
    resampled['ask_price'] = df.ask_price.resample('1S').mean()
    resampled['ask_size'] = df.ask_size.resample('1S').sum()
    resampled = resampled.join(turnover_and_mtp_data(asset,date)['mtp'])
    resampled['mtp'].iloc[0] = (resampled['bid_price'].iloc[0] + resampled['ask_price'].iloc[0])/2 # manuallly set mtp at t=0 
    return resampled.dropna()

#spy_1 = get_historic_quotes('SPY','2020-06-11')
#print(spy_1)




def get_training_data(asset,date,forecast_window = 20,lag = 5,plot_results = False):
    #1. rolling_sum(oi/bid_ask_spread) over lag window, 2. rolling_sum(oir/bid_ask_spread), 3. mpb/bid_ask_spread
    qdf = get_historic_quotes(asset,date)
    spread = qdf['ask_price'] - qdf['bid_price']
    qdf['dVb'] = qdf['bid_size'].copy()
    qdf.loc[qdf['bid_price'] < qdf['bid_price'].shift(1),'dVb'] = 0
    qdf.loc[qdf['bid_price'] == qdf['bid_price'].shift(1),'dVb'] = qdf['bid_size'] - qdf['bid_size'].shift(1)
    qdf['dVa'] = qdf['ask_size'].copy()
    qdf.loc[qdf['ask_price'] > qdf['ask_price'].shift(1),'dVa'] = 0
    qdf.loc[qdf['ask_price'] == qdf['ask_price'].shift(1),'dVa'] = qdf['ask_size'] - qdf['ask_size'].shift(1)
    imbalance = qdf['dVb'] - qdf['dVa']
    mid_price = (qdf['bid_price'] + qdf['ask_price'])/2
    mid_price_basis = qdf['mtp'] - 0.5*(mid_price.shift(1) + mid_price)
    qdf['mpb'] = mid_price_basis
    qdf['imbalance'] = imbalance
    ir = (qdf['dVb'] - qdf['dVa'])/(qdf['dVb'] + qdf['dVa'])
    qdf['imbalance_over_spread'] = imbalance/spread
    qdf['imbalance_ratio_over_spread'] = ir/spread
    qdf['mpb_over_spread'] = mid_price_basis/spread
    qdf['forecast_mid_price'] = (mid_price.iloc[::-1].rolling(forecast_window).sum() - forecast_window*mid_price)/forecast_window
    qdf = qdf.drop(['dVb','dVa','mtp'],axis = 1)
    if plot_results: 
        plt.figure(0).suptitle('mpb vs actual price change')
        plt.scatter(x = qdf['mpb'],y =qdf['forecast_mid_price'])
        plt.figure(1).suptitle('order_imbalance')
        plt.plot(qdf['imbalance'].tolist(), color = 'red')
        plt.figure(2).suptitle('average price change')
        plt.plot(qdf['forecast_mid_price'].tolist(), color ='green')
        plt.figure(3).suptitle('mid price basis')
        plt.plot(qdf['mpb'].tolist(),color = 'orange')
        plt.show()
    return qdf.replace([np.inf, -np.inf], np.nan).dropna() # still have to drop the imbalance and mbp before feeeding to NN

def is_series_stationary(series,is_df = True):
    if is_df: series = series.to_numpy().reshape((-1,1))
    try: 
        adf_result = adfuller(series)
        kpss_result = kpss(series)
    except: 
        return False
    is_adf_stationary = adf_result[0] < adf_result[4]['1%']
    is_kpss_stationary = kpss_result[0] < kpss_result[3]['10%']
    if is_adf_stationary and is_kpss_stationary: 
        return True
    return False

def backtest_stationarity(asset,end_date,lookback,print_errors = False):
    logs = []
    num_days_mean_reverting = 0 
    daterange = backtest_daterange(end_date,lookback)
    backtest_len = len(daterange)
    for date in daterange:
        try: 
            results = get_training_data(asset,date)
        except: 
            if print_errors: print('error getting training_data for {}'.format(date))
            backtest_len -= 1 
            continue
        if is_series_stationary(results['imbalance']) and is_series_stationary(results['forecast_mid_price']):
            logs.append(results)
            num_days_mean_reverting += 1 
    print('{} found to be mean reverting {}% of the time from {} to {}'.format(asset,num_days_mean_reverting*100/backtest_len,daterange[0],daterange[-1]))    
    return logs 
        
    
def prepare_nn_data(df):
    df = df.replace([np.inf, -np.inf], np.nan).dropna()
    X = df[['imbalance_over_spread','imbalance_ratio_over_spread','mpb_over_spread']].to_numpy()
    Y = df['forecast_mid_price'].to_numpy().reshape((-1,1)) 
    return X,Y

In [ ]:
result = backtest_stationarity('GDX','2020-06-09',30)
print([datetime.datetime.strftime(df.index[0],'%Y-%m-%d') for df in result])

In [40]:
df = get_training_data('GDX','2020-05-11',forecast_window = 20)

X_train,Y_train = prepare_nn_data(df)
olsModel = sm.OLS(Y_train,X_train)
results = olsModel.fit()
#print(results.summary())


pred_df = get_training_data('GDX','2020-05-12',forecast_window = 20)

X_test,Y_test = prepare_nn_data(pred_df)

ypred = pd.Series(results.predict(X_test))
ypred.index = pred_df.index


pred_df['mpb_pred'] = ypred
pred_df.loc[pred_df['mpb_pred'] < 0,'mpb_pred'] = -1 # sell signal
pred_df.loc[pred_df['mpb_pred'] > 0,'mpb_pred'] = 1 # buy signal
pred_df['buy_price'] = 0 
pred_df['sell_price'] = 0 
pred_df['shifted_bid'] = pred_df['bid_price'].shift(-20)
pred_df['shifted_ask'] = pred_df['ask_price'].shift(-20)

pred_df.loc[pred_df['mpb_pred'] < 0,'buy_price'] = pred_df.loc[pred_df['mpb_pred'] < 0,'bid_price']
pred_df.loc[pred_df['mpb_pred'] > 0,'buy_price'] = pred_df.loc[pred_df['mpb_pred'] > 0,'ask_price']

pred_df.loc[pred_df['mpb_pred'] > 0,'sell_price'] = pred_df.loc[pred_df['mpb_pred'] > 0,'shifted_bid']
pred_df.loc[pred_df['mpb_pred'] < 0,'sell_price'] = pred_df.loc[pred_df['mpb_pred'] < 0,'shifted_ask'] 

pred_df['net_pl'] = pred_df['sell_price']  - pred_df['buy_price'] 
pred_df.loc[pred_df['mpb_pred'] < 0,'net_pl'] *= -1

print(pred_df)
print('total P/L: {}'.format(pred_df['net_pl'].sum()))


#nn = network()
#nn.train_model(X_test,Y_test,num_epochs = 5)





                           bid_price  bid_size  ask_price  ask_size  \
sip_timestamp                                                         
2020-05-12 08:22:58-04:00  34.160000        12  34.223333         8   
2020-05-12 08:27:32-04:00  34.060000         8  34.150000        10   
2020-05-12 08:36:31-04:00  34.078000        41  34.120000         5   
2020-05-12 09:25:44-04:00  34.221429        23  34.260000        73   
2020-05-12 09:25:56-04:00  34.260000         7  34.280000        51   
...                              ...       ...        ...       ...   
2020-05-12 09:52:28-04:00  34.273415       625  34.281951      1894   
2020-05-12 09:52:31-04:00  34.270000       545  34.280000      1633   
2020-05-12 09:52:32-04:00  34.270000       209  34.280000       635   
2020-05-12 09:52:35-04:00  34.270000        21  34.280000       302   
2020-05-12 09:52:40-04:00  34.260000       571  34.270000       231   

                                    mpb  imbalance  imbalance_over_spread  \

In [ ]:
mdf= api.polygon.historic_agg_v2('GDX',1,'minute',_from= '2020-06-09',to='2020-06-09').df
mdf = mdf.join(df['mpb']).resample('1S')
print(mdf)

In [47]:
x = LpVariable("x",0,100)
y =LpVariable("y",0,90)
z = LpVariable("z",0,70)

problem = LpProblem('contracts',LpMinimize)
problem += x + y + z == 150 
problem += 500*x + 450*y + 450*z + 12000
status = problem.solve()
print(value(x),value(y),value(z),)


0.0 90.0 60.0
